# <p style="text-align: center;">MIS 382N: Advanced Predictive Modeling</p>
# <p style="text-align: center;">Assignment 5</p>
## <p style="text-align: center;">Total points: 50 </p>
## <p style="text-align: center;">Due: Mon, November 28</p>



Your homework should be written in a **Jupyter notebook**. Please submit **only one** ipynb file from each group, and include the names of all the group members. Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting.

# Problem 1 - Random Forest vs Boosting - Regression (15pts)

In this question, we will compare performance of different ensemble methods for regression problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html), [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor). Board game data set from DataQuest will be used (you can download data from Canvas: 'games.csv').

1. (1) Load the data, (2) remove duplicate rows, (3) remove features of type string (object in pandas), and (4) replace missing values by mean of each column. Then, partition data into features (X) and the target label (y) for regression task. We want to predict the *average_rating*. Use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split data into training and testing: test_size=0.33, random_state=42. (1pt)

2. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) to predict average_rating. Find the best parameters (including *n_estimators*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report the accuracy of your model in terms of RMSE. (4pts)

3. Use [Gradient Boosting Regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) (GBR), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor) for predicting the targets. Again, find the best parameters (including *n_estimators,* and* learning_rate*), and report corresponding RMSE for each algorithm. (8pts)

4. Which model did you expect to be more accurate in predicting the targets? Why? Did your observation match this expectation? (2pts)

In [64]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import (RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor)
from math import sqrt

#### 1) (1) Load the data, (2) remove duplicate rows, (3) remove features of type string (object in pandas), and (4) replace missing values by mean of each column. Then, partition data into features (X) and the target label (y) for regression task. We want to predict the average_rating. Use train_test_split to split data into training and testing: test_size=0.33, random_state=42. 

In [57]:
# Importing the data
data = pd.read_csv('games.csv')
print "Rows,columns=",data.shape
data[:5]

Rows,columns= (81312, 20)


,id,type,name,yearpublished,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,users_rated,average_rating,bayes_average_rating,total_owners,total_traders,total_wanters,total_wishers,total_comments,total_weights,average_weight
0,12333,boardgame,Twilight Struggle,2005.0,2.0,2.0,180.0,180.0,180.0,13.0,20113,8.33774,8.22186,26647,372,1219,5865,5347,2562,3.4785
1,120677,boardgame,Terra Mystica,2012.0,2.0,5.0,150.0,60.0,150.0,12.0,14383,8.28798,8.14232,16519,132,1586,6277,2526,1423,3.8939
2,102794,boardgame,Caverna: The Cave Farmers,2013.0,1.0,7.0,210.0,30.0,210.0,12.0,9262,8.28994,8.06886,12230,99,1476,5600,1700,777,3.7761
3,25613,boardgame,Through the Ages: A Story of Civilization,2006.0,2.0,4.0,240.0,240.0,240.0,12.0,13294,8.20407,8.05804,14343,362,1084,5075,3378,1642,4.1590
4,3076,boardgame,Puerto Rico,2002.0,2.0,5.0,150.0,90.0,150.0,12.0,39883,8.14261,8.04524,44362,795,861,5414,9173,5213,3.2943


In [58]:
# Removing duplicate rows
data = data.drop_duplicates()
print "After removing duplicates:",data.shape,"\n"

# Removing columns of string object
data = data[data.dtypes[data.dtypes != "object"].index]
print "After removing columns of string object:",data.shape,"\n"

# Replace missing values by mean
data = data.fillna(data.mean())
print "Missing values present?:",data.isnull().values.any()

# Splitting into train and test
X = data.ix[:, data.columns != 'average_rating']
y = data['average_rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

After removing duplicates: (79463, 20) 

After removing columns of string object: (79463, 18) 

Missing values present?: False


#### 2) Use a Random Forest to predict average_rating. Find the best parameters (including n_estimators) using GridSearchCV. Report the accuracy of your model in terms of RMSE.

In [59]:
# Setting up the model
parameters = {'n_estimators':[5,10,15],'max_depth':[None,5,10,15],'min_samples_split':[2,.1,.2],'min_samples_leaf':[2,.01,.02]}
rf_regressor = RandomForestRegressor()

# Optimizing parameters
grid_search_model = GridSearchCV(rf_regressor,parameters)

In [61]:
# Fitting the model on the data
grid_search_model.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [5, 10, 15], 'min_samples_split': [2, 0.1, 0.2], 'max_depth': [None, 5, 10, 15], 'min_samples_leaf': [2, 0.01, 0.02]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [62]:
# Printing out the optimal parameters
grid_search_model.best_params_

{'max_depth': 15,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 15}

In [66]:
# Predicting on test
pred_test = grid_search_model.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, pred_test))
print "RMSE using RF: ",rmse

RMSE using RF:  1.00461088488


#### 3) Use Gradient Boosting Regressor (GBR), and AdaBoost for predicting the targets. Again, find the best parameters (including n_estimators, and learning_rate), and report corresponding RMSE for each algorithm

##### Gradient boosting

In [73]:
# Setting up the model
parameters_gb = {'loss':('ls','huber', 'quantile'),'learning_rate': [.05,.1,.15],'n_estimators':[50,100,150],'max_depth':[2,3,5],'min_samples_split':[2,.1,.2],'min_samples_leaf':[1,.01,.02]}
gb_regressor = GradientBoostingRegressor()

# Optimizing parameters
grid_search_model_gb = GridSearchCV(gb_regressor,parameters_gb)

In [74]:
# Fitting the model on the data
grid_search_model_gb.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'loss': ('ls', 'huber', 'quantile'), 'learning_rate': [0.05, 0.1, 0.15], 'min_samples_leaf': [1, 0.01, 0.02], 'n_estimators': [50, 100, 150], 'min_samples_split': [2, 0.1, 0.2], 'max_depth': [2, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [75]:
# Printing out the optimal parameters
grid_search_model_gb.best_params_

{'learning_rate': 0.15,
 'loss': 'huber',
 'max_depth': 5,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 150}

In [76]:
# Predicting on test
pred_test = grid_search_model_gb.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, pred_test))
print "RMSE using GBR: ",rmse

RMSE using GBR:  0.992558014884


##### Adaboost

In [78]:
# Setting up the model
parameters_ada = {'loss':('linear','square'),'learning_rate': [.5,1,1.5],'n_estimators':[20,50,100]}
ada_regressor = AdaBoostRegressor()

# Optimizing parameters
grid_search_model_ada = GridSearchCV(ada_regressor,parameters_ada)

In [79]:
# Fitting the model on the data
grid_search_model_ada.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [20, 50, 100], 'loss': ('linear', 'square'), 'learning_rate': [0.5, 1, 1.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [80]:
# Printing out the optimal parameters
grid_search_model_ada.best_params_

{'learning_rate': 0.5, 'loss': 'linear', 'n_estimators': 50}

In [81]:
# Predicting on test
pred_test = grid_search_model_ada.predict(X_test)
rmse = sqrt(mean_squared_error(y_test, pred_test))
print "RMSE using Adaboost: ",rmse

RMSE using Adaboost:  1.1366529744


#### 4) Which model did you expect to be more accurate in predicting the targets? Why? Did your observation match this expectation?

Boosting is often more accurate than random forests since it reduces both bias and variance. Since Adaboost is a variation of gradient boosting that upweighs misclassified points, we expected it to perform the best.

In this scenario, GBM performed the best possibly because the Adaboost algorithm overfitted.

# Problem 2 - Random Forest vs Boosting - Classification (15 pts)
In this question, we will compare performance of different ensemble methods for classification problems: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier). [Spam Classification Data](https://archive.ics.uci.edu/ml/datasets/Spambase) of UCI will be used (you can download data from Canvas: 'spam_uci.csv'). Don't worry about column names. The last column represents target label, 1 if spam and zero otherwise.

1. Load the data and partition it into features (X) and the target label (y) for classification task. Then, use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split data into training and testing: test_size=0.33, random_state=42. (1pt)

2. Use a [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to classify whether an email is spam. Find the best parameters (including *n_estimators* and *criterion*) using [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Report your testing accuracy ([accuracy_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)) and [roc_auc_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score). You will need [predict_proba](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.predict_proba) for roc_auc_score. (4pts)

3. Use [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) for the spam classification problem. Again, find the best parameters (including *n_estimators, learning_rate,* and *max_depth (GBDT only)*), and report corresponding accuracy_score and roc_auc_score on the test data for each algorithm. (8pts)

4. Point out one advantage and one disadvantage of Random Forest compared to GBDT (2pts)

In [119]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,GridSearchCV)
from sklearn.metrics import (accuracy_score,roc_auc_score)
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier)
from IPython.display import display

#### 1) Load the data and partition it into features (X) and the target label (y) for classification task. Then, use train_test_split to split data into training and testing: test_size=0.33, random_state=42

In [83]:
# Importing the data
spam_uci = pd.read_csv('spam_uci.csv')
spam_uci[:5]

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,48,49,50,51,52,53,54,55,56,57
0,0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [89]:
# Splitting into train and test
X = spam_uci.ix[:, spam_uci.columns != '57']
y = spam_uci['57']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#### 2) Use a Random Forest to classify whether an email is spam. Find the best parameters (including n_estimators and criterion) using GridSearchCV. Report your testing accuracy (accuracy_score) and roc_auc_score. You will need predict_proba for roc_auc_score.

In [93]:
# Setting up the model
parameters = {'n_estimators':[5,10,15],'criterion':('gini','entropy'),'max_depth':[None,5,10,15],'min_samples_split':[2,.1,.2],'min_samples_leaf':[2,.01,.02]}
rf_classifier = RandomForestClassifier()

# Optimizing parameters
grid_search_model_class = GridSearchCV(rf_classifier,parameters)

In [94]:
# Fitting the model on the data
grid_search_model_class.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [5, 10, 15], 'min_samples_split': [2, 0.1, 0.2], 'criterion': ('gini', 'entropy'), 'max_depth': [None, 5, 10, 15], 'min_samples_leaf': [2, 0.01, 0.02]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [95]:
# Printing out the optimal parameters
grid_search_model_class.best_params_

{'criterion': 'gini',
 'max_depth': 15,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 10}

In [124]:
# Predicting on test
pred_test = grid_search_model_class.predict(X_test)

# Getting performance stats
display(pd.crosstab(pred_test,y_test,rownames = ["Predicted_class"], colnames = ["Actual_class"]))
print "The accuracy is: ", accuracy_score(y_test, pred_test),"\n"
print "The ROC AUC is: ",roc_auc_score(y_test, grid_search_model_class.predict_proba(X_test)[:,1])

Actual_class,0,1
Predicted_class,,
0,883,1
1,3,632


The accuracy is:  0.997366688611 

The ROC AUC is:  0.999987518677


#### 3) Use Gradient Boosting Decision Tree (GBDT), and AdaBoost for the spam classification problem. Again, find the best parameters (including n_estimators, learning_rate, and max_depth (GBDT only)), and report corresponding accuracy_score and roc_auc_score on the test data for each algorithm.

##### Gradient boosting

In [125]:
# Setting up the model
parameters_gb = {'loss':('deviance','exponential'),'learning_rate': [.05,.1,.15],'n_estimators':[50,100,150],'max_depth':[2,3,5],'min_samples_split':[2,.1,.2],'min_samples_leaf':[1,.01,.02]}
gb_classifier = GradientBoostingClassifier()

# Optimizing parameters
grid_search_model_class_gb = GridSearchCV(gb_classifier,parameters_gb)

In [126]:
# Fitting the model on the data
grid_search_model_class_gb.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'loss': ('deviance', 'exponential'), 'learning_rate': [0.05, 0.1, 0.15], 'min_samples_leaf': [1, 0.01, 0.02], 'n_estimators': [50, 100, 150], 'min_samples_split': [2, 0.1, 0.2], 'max_depth': [2, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [127]:
# Printing out the optimal parameters
grid_search_model_class_gb.best_params_

{'learning_rate': 0.05,
 'loss': 'deviance',
 'max_depth': 2,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 50}

In [128]:
# Predicting on test
pred_test = grid_search_model_class_gb.predict(X_test)

# Getting performance stats
display(pd.crosstab(pred_test,y_test,rownames = ["Predicted_class"], colnames = ["Actual_class"]))
print "The accuracy is: ", accuracy_score(y_test, pred_test),"\n"
print "The ROC AUC is: ",roc_auc_score(y_test, grid_search_model_class_gb.predict_proba(X_test)[:,1])

Actual_class,0,1
Predicted_class,,
0,884,0
1,2,633


The accuracy is:  0.998683344305 

The ROC AUC is:  0.998871331828


##### Adaboost

In [129]:
# Setting up the model
parameters_ada = {'algorithm':('SAMME','SAMME.R'),'learning_rate': [.5,1,1.5],'n_estimators':[20,50,100]}
ada_classifier = AdaBoostClassifier()

# Optimizing parameters
grid_search_model_class_ada = GridSearchCV(ada_classifier,parameters_ada)

In [130]:
# Fitting the model on the data
grid_search_model_class_ada.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [20, 50, 100], 'learning_rate': [0.5, 1, 1.5], 'algorithm': ('SAMME', 'SAMME.R')},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [131]:
# Printing out the optimal parameters
grid_search_model_class_ada.best_params_

{'algorithm': 'SAMME', 'learning_rate': 0.5, 'n_estimators': 20}

In [134]:
# Predicting on test
pred_test = grid_search_model_class_ada.predict(X_test)

# Getting performance stats
display(pd.crosstab(pred_test,y_test,rownames = ["Predicted_class"], colnames = ["Actual_class"]))
print "The accuracy is: ", accuracy_score(y_test, pred_test),"\n"
print "The ROC AUC is: ",roc_auc_score(y_test, grid_search_model_class_ada.predict_proba(X_test)[:,1])

Actual_class,0,1
Predicted_class,,
0,884,0
1,2,633


The accuracy is:  0.998683344305 

The ROC AUC is:  0.998871331828


#### 4) Point out one advantage and one disadvantage of Random Forest compared to GBDT

Advantages - Random forests are much easier to tune, have a lower tendency to overfit and can be parallelized.

Disadvantages - With correct tuning parameters, GBDT generally gives better accuracy when compared to random forest.

# Question 3 - Matrix Factorization for Rating Prediction (20pts)

The movielens dataset contains 1 million movie ratings from several thousand users. We will be using *k*-rank matrix factorization to estimate this dataset as the product $X=UV^T$, where *U* and *V* have only $k$ columns.

1) You can download the movielens 1M dataset from https://datahub.io/dataset/movielens, but for this problem use the data available on Canvas. It has been split into training and test sets, and converted to matrix format where the rows correspond to users and the columns to movies. Note that most of the entries are NaNs, indicating that these ratings are missing. An extra file, lens1m_361M_titles.csv, has been added so you can check out specific movies if you're curious.

2) Scikit-learn is a little behind for recommender systems, and doesn't have any method to factorize matrices with missing data. Which means you get to code it! Slide 22 of the 'apa large scale learning' lecture notes has the equations for stochastic gradient descent on *U* and *V*. You will have to:
* Set up initial guesses for the *U* and *V* matrices. I suggest small random values.
* Find a suitable learning rate for the descent. A learning rate that is too large will probably blow up, like in HW3 problem 1.
* Come up with a stopping policy
* Code the descent algorithm (5 pts)

3) Using your SGD algorithm, apply 2-rank matrix factorization on the filled training matrix. Calculate the RMSE of this model on the training data and on the test data (separately). The optimal score on the training data is around .86 RMSE; your version of gradient descent must go at least below .91 RMSE. (5 pts)

4) You should notice some overfitting. Because matrix factorization learns separate scores for each movie, a movie with very few reviews may be easily overfit. You may want to only predict ratings when you have enough information to reach a good conclusion. Recalculate the RMSE on the test data, specifically for movies with at least 50 reviews (don't retrain the models). Also report the percent of movies that are still included (after cutting those with < 50 reviews), and the percent of test ratings that are still included. (5 pts)

5) Repeat steps 3 and 4 with 5-rank factorization. Display training and test RMSE. (5 pts)

Hints:  
The numpy function *nanmean* is helpful for RMSE calculation.  
The descent algorithm will probably run for at least several minutes.

In [3]:
import numpy as np
import pandas as pd
#titles = pd.read_csv('lens1m_361M_titles.csv')
test_X = np.load('lens1m_361M_test.npy')
train_X = np.load('lens1m_361M_train.npy')

1. initially filled u and v with random value with the help of random generator library python
2. we did not choose a very low value of alpha ( learning rate ) nor a very high value of alpha. We decided to go with 0.002 for 2-rank and 0.01 for 5-rank since it seems to be low, but not low enough and high, but not high enough. 
3. our stopping policy is when the rmse goes below 0.91 ( which is the worst case value ) or when the number of iterations have gone beyond 10, whichever is lower. We chose 10 iterations because it seemed to be optimal and manages to converge to give good enough rmse. 

In [5]:
def sgd(x,u,v,k,alpha):
    for i in range(0,10):
        for j in range(len(x)):
            for jter in range(len(x[j])):
                if x[j][jter] > 0:
                    for kter in range(0,k):
                        integral_1 = 2 * alpha * (x[j][jter] - np.dot(u[j,:],v[:,jter])) * v[kter][jter]
                        integral_2 = 2 * alpha * (x[j][jter] - np.dot(u[j,:],v[:,jter])) * u[j][kter]
                        u[j][kter] = u[j][kter] + integral_1
                        v[kter][jter] = v[kter][jter] + integral_2
        
        new_x = np.dot(u,v)
        if np.sqrt(np.nanmean(np.square(np.subtract(train_X,new_x)))) < 0.91:
            return u,v
        else:
            continue
    
    return u,v
    

In [27]:
u = np.random.rand(len(train_X),2)
v = np.random.rand(len(train_X[0]),2)
v = v.T

In [28]:
new_u,new_v = sgd(train_X,u,v,2,0.002)

In [29]:
new_x = np.dot(new_u,new_v)

In [30]:
np.sqrt(np.nanmean(np.square(np.subtract(train_X,new_x))))
#rmse training for 2-rank factorization 

0.90914050466342311

In [31]:
sum_rmse = []
test_rating_count = 0
for i in range(3952):
    if np.count_nonzero(~np.isnan(test_X[:,i])) >= 50:
        test_rating_count+=np.count_nonzero(~np.isnan(test_X[:,i]))
        sum_rmse.append(np.square(np.subtract(test_X[:,i],new_x[:,i])))

In [32]:
rmse_test = np.sqrt(np.nanmean(sum_rmse))

In [33]:
#rmse test for 2-rank factorization with greater than 50 movie reviews
rmse_test

0.91433753482028801

In [13]:
#percent of movies retained :

1.0*len(sum_rmse)*100/3952

30.794534412955464

In [4]:
index = []
test_rating_count = 0
for i in range(3952):
    if np.count_nonzero(~np.isnan(test_X[:,i])) >= 50:
        index.append(i)

In [6]:
#test rating count retained as a fraction of total ratings : 
1.0*(np.count_nonzero(~np.isnan(test_X[:,index])))/np.count_nonzero(~np.isnan(test_X))

0.8119394927065317

repeating with rank 5 factorization : 
    

In [15]:
u = np.random.rand(len(train_X),5)
v = np.random.rand(len(train_X[0]),5)
v = v.T

In [16]:
new_u,new_v = sgd(train_X,u,v,5,0.01)


In [20]:
new_x = np.dot(new_u,new_v)
print("training rmse", np.sqrt(np.nanmean(np.square(np.subtract(train_X,new_x)))))


('training rmse', 0.90606213063938978)


In [18]:
sum_rmse = []
test_rating_count = 0
for i in range(3952):
    if np.count_nonzero(~np.isnan(test_X[:,i])) >= 50:
        test_rating_count+=np.count_nonzero(~np.isnan(test_X[:,i]))
        sum_rmse.append(np.square(np.subtract(test_X[:,i],new_x[:,i])))
rmse_test = np.sqrt(np.nanmean(sum_rmse))

In [21]:
#test rmse
print("test rmse",rmse_test)

('test rmse', 0.93059455524253476)
